In [1]:
import pandas as pd

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = '../../data/'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
SKILLCORNER_RAW_PATH = DATA_PATH + 'skillcorner_raw/'
XT_PLOT_PATH = DATA_PATH + 'smoothed_xt.csv'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'
WYSCOUT_TO_SKILLCORNER = DATA_PATH + 'wyscout2skillcorner.csv'

## Load Data

In [3]:
match_info = pd.read_csv(MATCH_IDS_PATH)
match_info = match_info.dropna(subset=['wyscout', 'skillcorner'])
match_info['skillcorner'] = match_info['skillcorner'].astype(int)

In [4]:
match_info.head(5)

,wyscout,skillcorner,date,home,result,away
8,5414111,952209,2023-03-24,France,4-0,Netherlands
10,5414113,726849,2023-03-24,Czech Republic,3-1,Poland
12,5414115,722109,2023-03-24,Sweden,0-3,Belgium
25,5414127,726848,2023-03-26,England,2-0,Ukraine
33,5414135,952208,2023-03-27,Republic of Ireland,0-1,France


In [5]:
wyscout_id = match_info['wyscout'].values[0]
skillcorner_id = match_info['skillcorner'].values[0]

### wyscout

In [6]:
wyscout_data = wyscout_to_df(WYSCOUT_PATH + str(wyscout_id) + ".json")
with pd.option_context('display.max_columns', None):
    display(wyscout_data.head(3))

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
0,1587492614,5414111,1H,0,1,00:00:01.912,3.912638,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",51,50,664,Netherlands,4-2-3-1,4418,France,4-3-3,118,M. Depay,CF,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,1.587493e+09,110.275779,[],43.0,0.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1587492615,5414111,1H,0,5,00:00:05.782,7.782226,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",42,83,664,Netherlands,4-2-3-1,4418,France,4-3-3,625,M. de Roon,RDMF,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,1.587493e+09,110.275779,[],43.0,1.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1587492616,5414111,1H,0,8,00:00:08.951,10.951834,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",29,60,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,1.587493e+09,110.275779,[],43.0,2.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### skillcorner

In [7]:
metadata = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_metadata.csv")
tracking_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_tracking.csv")
lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")
play_direction_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_play_direction.csv")

In [8]:
print("Metadata")
display(metadata.head(3))
print("Tracking")
display(tracking_df.head(3))
print("Lineup")
display(lineup_df.head(3))
print("Play Direction")
display(play_direction_df)


Metadata


,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,home_team_jersey_color,away_team_jersey_color,home_team_number_color,away_team_number_color,home_team_coach,away_team_coach,pitch_name,pitch_length,pitch_width,provider,fps
0,952209,03/24/2023 19:45,European Championship Qualifiers,2023/2024,France,Netherland,4,0,#fafafa,#ff824c,#000000,#000000,Didier Deschamps,Frank De Boer,Stade de France,105.0,68.0,SkillCorner,10


Tracking


,match_id,half,frame_id,timestamp,object_id,x,y,z,extrapolated
0,952209,1,20,0,-1,0.12,-0.19,0.12,False
1,952209,1,20,0,7604,-39.44,-0.21,0.00,True
2,952209,1,20,0,16320,-16.05,-7.18,0.00,False


Lineup


,match_id,team_name,player_id,player_first_name,player_last_name,player_shirt_number,player_position,player_birthdate,start_time,end_time,yellow_card,red_card,injured,goal,own_goal
0,952209,Netherland,14397,Tyrell,Malacia,16,Right Wing Back,1999-08-17,01:26:59,NaN,0,0,False,0,0
1,952209,Netherland,12654,Donyell,Malen,7,Right Winger,1999-01-19,01:06:22,NaN,0,0,False,0,0
2,952209,Netherland,2548,Davy,Klaassen,14,Attacking Midfield,1993-02-21,01:06:22,NaN,0,0,False,0,0


Play Direction


,match_id,team_name,half,play_direction
0,952209,France,1,BOTTOM_TO_TOP
1,952209,Netherland,1,TOP_TO_BOTTOM
2,952209,France,2,TOP_TO_BOTTOM
3,952209,Netherland,2,BOTTOM_TO_TOP


In [9]:
a = tracking_df[tracking_df['object_id'] == 7604]
display(a[a['half'] == 1][['x', 'y']].mean())
display(a[a['half'] == 2][['x', 'y']].mean())

x   -38.547018
y     0.604576
dtype: float64

x    41.171923
y    -0.061672
dtype: float64

In [10]:
a = tracking_df[tracking_df['object_id'] == 4777]
display(a[a['half'] == 1][['x', 'y']].mean())
display(a[a['half'] == 2][['x', 'y']].mean())


x    38.239516
y    -0.025721
dtype: float64

x   -35.559305
y     0.312805
dtype: float64

# Pass Events

In [11]:
passes_df = wyscout_data[wyscout_data['type.primary'] == 'pass']

# Filter out inaccurate passes
accurate_only = True
if accurate_only:
    passes_df = passes_df[passes_df['pass.accurate'] == True]
    
# Filter out passes with invalid recipient id
passes_df = passes_df[passes_df['pass.recipient.id'] != 0]

print(passes_df.shape)

(894, 93)


In [12]:
passes = passes_df.loc[:,['videoTimestamp', 'matchPeriod', 'player.id', 'pass.recipient.id', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']]
passes['matchPeriod'] = passes['matchPeriod'].apply(lambda x: int(x.split('H')[0]))
passes['videoTimestamp'] = passes['videoTimestamp'].astype(float)
passes['pass.recipient.id'] = passes['pass.recipient.id'].astype(int)
passes['pass.endLocation.x'] = passes['pass.endLocation.x'].astype(int)
passes['pass.endLocation.y'] = passes['pass.endLocation.y'].astype(int)

In [13]:
passes.head()

,videoTimestamp,matchPeriod,player.id,pass.recipient.id,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,3.912638,1,118,625,51,50,42,83
1,7.782226,1,625,476361,42,83,29,60
2,10.951834,1,476361,370,29,60,25,28
3,15.151013,1,370,476361,25,28,25,51
5,19.930736,1,476361,370,32,50,30,17


# Map Wyscout IDs to SkillCorner IDs

In [14]:
wyscout2skillcorner = pd.read_csv(WYSCOUT_TO_SKILLCORNER).drop(columns='id')

In [15]:
passes.rename(columns={'player.id': 'player.id.wyscout', 'pass.recipient.id': 'pass.recipient.id.wyscout'}, inplace=True)

In [16]:
passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy', 'team_name_sk']], left_on='player.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'player.id.skillcorner'}, inplace=True)
passes.rename(columns={'team_name_sk': 'team.name'}, inplace=True)

passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy']], left_on='pass.recipient.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'pass.recipient.id.skillcorner'}, inplace=True)

In [17]:
cols = ['videoTimestamp', 'matchPeriod', 'team.name', 'player.id.wyscout', 'player.id.skillcorner', 'pass.recipient.id.wyscout', 'pass.recipient.id.skillcorner', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']
passes = passes.loc[:,cols]

In [18]:
passes.head()

,videoTimestamp,matchPeriod,team.name,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,3.912638,1,Netherland,118,7387,625,7017,51,50,42,83
1,7.782226,1,Netherland,625,7017,476361,14407,42,83,29,60
2,10.951834,1,Netherland,476361,14407,370,10738,29,60,25,28
3,15.151013,1,Netherland,370,10738,476361,14407,25,28,25,51
4,19.930736,1,Netherland,476361,14407,370,10738,32,50,30,17


# Play Direction

In [19]:
play_direction_dict = play_direction_df.drop(columns='match_id').set_index(['team_name', 'half']).to_dict()['play_direction']
play_direction_dict

{('France', 1): 'BOTTOM_TO_TOP',
 ('Netherland', 1): 'TOP_TO_BOTTOM',
 ('France', 2): 'TOP_TO_BOTTOM',
 ('Netherland', 2): 'BOTTOM_TO_TOP'}

In [20]:
player_team_dict = lineup_df[['player_id', 'team_name']].set_index('player_id').to_dict()['team_name']

In [21]:
def get_play_direction(row):
    team_name = player_team_dict[row['player.id.skillcorner']]
    return play_direction_dict[(team_name, row['matchPeriod'])]

In [22]:
passes['play_direction'] = passes.apply(get_play_direction, axis=1)

# Compute ΔxT

In [23]:
xt_table = pd.read_csv(XT_PLOT_PATH)

In [24]:
cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

def get_xt_index(x, y, direction):
    x_adjusted = x if direction == 1 else 100 - x
    y_adjusted = y if direction == 1 else 100 - y
    x_index = int(min(x_adjusted // cell_width, xt_table.shape[1] - 1))
    y_index = int(min(y_adjusted // cell_height, xt_table.shape[0] - 1))
    return x_index, y_index

def get_xt_value(x, y, direction):
    x_index, y_index = get_xt_index(x, y, direction)
    return xt_table.iat[y_index, x_index]

In [25]:
passes['start_xt_right'] = passes.apply(lambda row: get_xt_value(row['location.x'], row['location.y'], 1), axis=1)
passes['end_xt_right'] = passes.apply(lambda row: get_xt_value(row['pass.endLocation.x'], row['pass.endLocation.y'], 1), axis=1)
passes['start_xt_left'] = passes.apply(lambda row: get_xt_value(row['location.x'], row['location.y'], -1), axis=1)
passes['end_xt_left'] = passes.apply(lambda row: get_xt_value(row['pass.endLocation.x'], row['pass.endLocation.y'], -1), axis=1)

# Calculate dxt for both directions
passes['dxt_right'] = passes['end_xt_right'] - passes['start_xt_right']
passes['dxt_left'] = passes['end_xt_left'] - passes['start_xt_left']

# Add a new column 'dxt' based on the play_direction condition
passes['dxt'] = passes.apply(lambda row: row['dxt_left'] if row['play_direction'] == 'TOP_TO_BOTTOM' else row['dxt_right'], axis=1)

columns_to_drop = ['start_xt_right', 'start_xt_left', 'end_xt_right', 'end_xt_left', 'dxt_right', 'dxt_left']
passes = passes.drop(columns=columns_to_drop)

In [26]:
passes.sample(5)

,videoTimestamp,matchPeriod,team.name,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,dxt
836,5175.044215,2,Netherland,491458,24466,33,4777,37,62,19,54,BOTTOM_TO_TOP,-0.002425
680,3949.243347,2,Netherland,370,10738,491458,24466,60,24,61,64,BOTTOM_TO_TOP,0.002310
84,591.678972,1,Netherland,134102,7863,535816,33512,26,14,36,22,TOP_TO_BOTTOM,-0.010534
336,1684.071048,1,France,209117,277,352988,11516,27,22,19,14,BOTTOM_TO_TOP,-0.000929
807,4970.891506,2,Netherland,38,2106,116,3837,51,17,53,58,BOTTOM_TO_TOP,0.002048


# Player Locations

In [27]:
framerate = metadata.loc[0,'fps']

def videotime_to_frame(videotime):
    return int(videotime * framerate)

In [28]:
passes.insert(1, 'frame', passes['videoTimestamp'].apply(videotime_to_frame))
passes.drop(columns='videoTimestamp', inplace=True)

In [ ]:
tracking_df.rename(columns={'frame_id': 'frame'}, inplace=True)
tracking_df.set_index('frame', inplace=True)
passes.set_index('frame', inplace=True)

joined_df = passes.join(tracking_df, how='inner', validate='one_to_many')

In [ ]:
columns_to_drop = ['match_id', 'half', 'timestamp']
existing_columns_to_drop = [col for col in columns_to_drop if col in joined_df.columns]
joined_df.drop(columns=existing_columns_to_drop, inplace=True)

columns_to_prefix = ['object_id', 'x', 'y', 'z']
prefix = 'tracking.'
joined_df.rename(columns={col: prefix + col for col in columns_to_prefix}, inplace=True)

In [31]:
joined_df.sample(5)

,matchPeriod,team.name,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,extrapolated
frame,,,,,,,,,,,,,,,,,
51637,2,Netherland,118,7387,38,2106,93,91,85,83,BOTTOM_TO_TOP,-0.005707,2106,42.65,-31.75,0.0,False
11824,1,France,480145,12218,134496,7604,32,19,10,40,BOTTOM_TO_TOP,-0.001608,6028,6.76,18.58,0.0,False
47960,2,France,525236,12229,393247,16320,39,68,34,84,TOP_TO_BOTTOM,0.001628,12246,-2.40,18.11,0.0,False
50146,2,Netherland,476361,14407,370,10738,44,75,41,31,BOTTOM_TO_TOP,-0.000750,6028,7.98,-24.65,0.0,False
9285,1,Netherland,370,10738,134102,7863,44,23,48,14,TOP_TO_BOTTOM,-0.003271,3837,-7.64,0.93,0.0,False


# Normalize Pitch Coordinates

In [32]:
pitch_length = metadata['pitch_length'].values[0]
pitch_width = metadata['pitch_width'].values[0]

In [33]:
start_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['location.x'], row['location.y'], pitch_length, pitch_width, row['play_direction']), 
    axis=1)
end_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['pass.endLocation.x'], row['pass.endLocation.y'], pitch_length, pitch_width, row['play_direction']), 
    axis=1)
joined_df[['location.x', 'location.y']] = start_locations.apply(pd.Series)
joined_df[['pass.endLocation.x', 'pass.endLocation.y']] = end_locations.apply(pd.Series)

# Identify Tracking Objects

In [34]:
def is_self(row):
    return row['player.id.skillcorner'] == row['tracking.object_id']

def is_teammate(row):
    if row['tracking.object_id'] == -1:
        return False
    player_team = player_team_dict[row['player.id.skillcorner']]
    tracking_player_team = player_team_dict[row['tracking.object_id']]
    return player_team == tracking_player_team

In [35]:
joined_df['tracking.is_self'] = joined_df.apply(is_self, axis=1)
joined_df['tracking.is_teammate'] = joined_df.apply(is_teammate, axis=1)

# Defender Responsibility

In [36]:
joined_df['responsibility'] = joined_df.apply(responsibility, axis=1, pass_length_factor=1)
joined_df['responsibility'] = np.where(joined_df['tracking.is_teammate'], 0, joined_df['responsibility'])

# Coordinate Normalization Evaluation

In [37]:
xt_table = pd.read_csv(XT_PLOT_PATH)

In [38]:
a = joined_df[joined_df['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']],), axis=1)
print(a.describe())

count    893.000000
mean       3.737207
std        2.249448
min        0.130206
25%        2.151520
50%        3.402867
75%        4.891735
max       21.157129
dtype: float64


In [39]:
first_period = joined_df[joined_df['matchPeriod'] == 1]
second_period = joined_df[joined_df['matchPeriod'] == 2]
a = first_period[first_period['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']],), axis=1)
b = second_period[second_period['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']]), axis=1)

print('First half')
display(a.describe())
print('Second half')
display(b.describe())

First half


count    517.000000
mean       3.514652
std        2.085332
min        0.130206
25%        2.085100
50%        3.284709
75%        4.585417
max       21.157129
dtype: float64

Second half


count    376.000000
mean       4.043221
std        2.426979
min        0.227633
25%        2.196651
50%        3.678401
75%        5.313445
max       16.513037
dtype: float64

In [40]:
first_period = joined_df[joined_df['matchPeriod'] == 1]
second_period = joined_df[joined_df['matchPeriod'] == 2]
a = first_period[first_period['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']],), axis=1)
b = second_period[second_period['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']]), axis=1)

print('First half')
display(a.describe())
print('Second half')
display(b.describe())

First half


count    517.000000
mean       3.514652
std        2.085332
min        0.130206
25%        2.085100
50%        3.284709
75%        4.585417
max       21.157129
dtype: float64

Second half


count    376.000000
mean       4.043221
std        2.426979
min        0.227633
25%        2.196651
50%        3.678401
75%        5.313445
max       16.513037
dtype: float64

In [41]:
extrapolated = joined_df[joined_df['extrapolated']]
not_extrapolated = joined_df[~joined_df['extrapolated']]
a = extrapolated[extrapolated['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']],), axis=1)
b = not_extrapolated[not_extrapolated['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']]), axis=1)

print('Extrapolated')
display(a.describe())
print('Not Extrapolated')
display(b.describe())

Extrapolated


count    99.000000
mean      4.216838
std       2.586048
min       0.386822
25%       2.137555
50%       3.893010
75%       5.429100
max      13.305496
dtype: float64

Not Extrapolated


count    794.000000
mean       3.677404
std        2.198397
min        0.130206
25%        2.152629
50%        3.378977
75%        4.747816
max       21.157129
dtype: float64

In [42]:
top_bot = joined_df[joined_df['play_direction'] == 'TOP_TO_BOTTOM']
bot_top = joined_df[joined_df['play_direction'] == 'BOTTOM_TO_TOP']
a = top_bot[top_bot['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']],), axis=1)
b = bot_top[bot_top['tracking.is_self']].apply(lambda row: np.linalg.norm([row['location.x'] - row['tracking.x'], row['location.y'] - row['tracking.y']]), axis=1)

print('Top to Bottom')
display(a.describe())
print('Bottom to Top')
display(b.describe())

Top to Bottom


count    402.000000
mean       3.739519
std        2.342433
min        0.130206
25%        2.125241
50%        3.358943
75%        4.985521
max       21.157129
dtype: float64

Bottom to Top


count    491.000000
mean       3.735314
std        2.172766
min        0.175085
25%        2.154688
50%        3.461547
75%        4.862048
max       16.513037
dtype: float64